### RatV Day3NSD recording info

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import subjects

sess = subjects.nsd.ratVday3[0] # add your subject session
print(sess.recinfo)

filename: /data/Clustering/sessions/RatV/RatVDay3NSD/RatV_Day3NSD_2021-10-07_08-10-12.xml 
# channels: 259
sampling rate: 30000
lfp Srate (downsampled): 1250



### Animal metadata

In [ ]:
from neuropy.core import animal

d = {'name':'hello','tag':'ser'}

an = animal.Animal(d) 


### Probe configuration
- Add text for details about the probes that were used

### A simple 8 shank probe with same layout across all shanks

In [ ]:
%matplotlib widget
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)

shanks = []
for i in range(8,16):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe2 = Probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plot_probe(prbgrp,channel_id=True)



### Create Probemap for spyking circus

In [ ]:
from neuropy.utils import probe_util

probe_util.write_spyking_circus(
    file=sess.filePrefix.with_suffix(".prb"), prb=sess.probegroup, combine_shanks=True
)


### Experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [ ]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
durations = datetime_data.nFrames/sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0,10136,13354,45527],
        "stop": [10135,13353,45526,47948],
        "label": ["pre", "maze",'post','re-maze'],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.save(sess.filePrefix.with_suffix(".paradigm.npy"))

In [ ]:
durations.cumsum()

In [ ]:
paradigm.durations/3600

### Neurons
- import spiketrains from Phy
- estimate neuron types such pyramidal, interneuron etc. 

### Importing spiketrains from Phy

### Single folder

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/home/bapung/Documents/ClusteringHub/spykcirc/RatV/RatVDay3NSD/RatV_Day3NSD_2021-10-07_08-10-12-1.GUI"
)
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id < 4] = "pyr"
neuron_type[neuron_type_id == 6] = "mua"
neuron_type[neuron_type_id == 8] = "inter"


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    neuron_type=neuron_type,
    shank_ids=np.array(shank_id).astype(int),
    metadata={'cluster_path':str(cluster_path)}
)

neurons.save(sess.filePrefix.with_suffix(".neurons"))

### Plot raster

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster

plot_raster(neurons,color='jet',add_vert_jitter=True)

### BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.save(sess.filePrefix.with_suffix(".mua.npy"))


In [ ]:
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

### Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

### Artifacts
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
from neuropy import analyses 

signal = sess.eegfile.get_signal([1,192])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7,edge_cutoff=2)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))

In [ ]:
%matplotlib widget
from neuropy import plotting
signal = sess.eegfile.get_signal([1])
plotting.plot_artifact_epochs(sess.artifact, signal)

### export for spyking circus

In [ ]:
from neuropy.io import SpykingCircusIO

SpykingCircusIO.write_epochs(sess.filePrefix.with_suffix('.dead'),sess.artifact)

### Ripple
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations

signal = sess.eegfile.get_signal()
ripple_epochs = oscillations.detect_ripple_epochs(
    signal, sess.probegroup, freq_band=(125,250),ignore_epochs=sess.artifact
)
ripple_epochs.save(sess.filePrefix.with_suffix(".ripple.npy"))

### Pbe

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


### Assign position data
- Position aligned using TTL signal in .dat/.eeg file (3 May 2022)
- Position cell has now been moved to hfuncs (4 May 2022)

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder,scale_factor=1)

In [ ]:
from hfuncs import position_alignment

ttl_signal = sess.eegfile.get_signal(258)
datetime_csv = sess.filePrefix.with_suffix('.datetime.csv')
position = position_alignment(opti_data,datetime_csv,ttl_signal,fs=30000)
# position.save(sess.filePrefix.with_suffix('.position'))

### Linearize position

In [ ]:
from neuropy.utils import position_util

for e in ['maze','re-maze']:
    maze = sess.paradigm[e].flatten()
    maze_pos = sess.position.time_slice(maze[0],maze[1])
    linear_pos = position_util.linearize_position(maze_pos)
    e = e.replace('-','')
    linear_pos.save(sess.filePrefix.with_suffix(f'.{e}.linear'))


In [ ]:
plt.plot(maze_pos.time,maze_pos.y)
plt.plot(linear_pos.time,linear_pos.x)